![image.png](https://images.aicrowd.com/raw_images/challenges/banner_file/1007/6edbed5d4a9dd4d28f16.jpg)

<h2><center>Starter Code for NLP Feature Engineering #2</center></h2>

<!-- <h6><center>Author : Shubhamai</center></h6> -->



<!-- --- -->



### What we are going to Learn

  - How to convert your text into numbers ?
  - How Bag of words, TF-IDF works ?
  - Testing and Submitting the Results to the Challenge. 

## About this Challanges

Now, this challange is very different form what we usually do in AIcrowd Blitz.In this challanges, the task is to generate features from a text data. Extracting features helps up to generate text embeddings will contains more useful information about the text. 


# Setup AIcrowd Utilities 🛠

We use this to bundle the files for submission and create a submission on AIcrowd. Do not edit this block.

In [ ]:
%%capture
!pip install -q -U aicrowd-cli
%load_ext aicrowd.magic

## How to use this notebook? 📝

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_DATASET_PATH` | Path to the file containing test data (The data will be available at `/data/` on aridhia workspace). This should be an absolute path.
`AICROWD_OUTPUTS_PATH` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#Install-packages-🗃) section to install the packages
- **Training your models**. All the code within the [Training phase ⚙️](#Training-phase-⚙️) section will be skipped during evaluation. **Please make sure to save your model weights in the assets directory and load them in the predictions phase section** 

## AIcrowd Runtime Configuration 🧷

Define configuration parameters. Please include any files needed for the notebook to run under `ASSETS_DIR`. We will copy the contents of this directory to your final submission file 🙂

The dataset is available under `/data` on the workspace.

In [ ]:
import os

# Please use the absolute for the location of the dataset.
# Or you can use relative path with `os.getcwd() + "test_data/test.csv"`
AICROWD_DATASET_PATH = os.getenv("DATASET_PATH", os.getcwd()+"/data/data.csv")
AICROWD_OUTPUTS_PATH = os.getenv("OUTPUTS_DIR", "")
AICROWD_ASSETS_DIR = os.getenv("ASSETS_DIR", "assets")

# Install packages 🗃

We are going to use many different libraries to demonstrate many idfferent techniques to convert text into numbers ( or more specifically vectors )

In [ ]:
!pip install --upgrade spacy rich gensim tensorflow scikit-learn
!python -m spacy download en_core_web_sm # Downloaing the model for english language will contains many pretrained preprocessing pipelines 

     |████████████████████████████████| 6.0 MB 4.8 MB/s 
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
     |████████████████████████████████| 24.8 MB 2.2 MB/s 
     |████████████████████████████████| 181 kB 36.4 MB/s 
     |████████████████████████████████| 10.1 MB 10.7 MB/s 
     |████████████████████████████████| 451 kB 41.2 MB/s 
     |████████████████████████████████| 628 kB 13.6 MB/s 
     |████████████████████████████████| 42 kB 693 kB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
   

# Define preprocessing code 💻

The code that is common between the training and the prediction sections should be defined here. During evaluation, we completely skip the training section. Please make sure to add any common logic between the training and prediction sections here.

In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
import random
from tqdm.notebook import tqdm

# Tensorflow 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score


# Word2vec Implementation
import spacy
nlp = spacy.load('en_core_web_sm', exclude=['tagger', 'ner', 'attribute_ruler', 'lemmatizer'])

from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

# To make things more beautiful! 
from rich.console import Console
from rich.table import Table
from rich.segment import Segment
from rich import pretty
pretty.install()

# Seeding everything for getting same results 
random.seed(42)
np.random.seed(42)

# function to display YouTube videos
from IPython.display import YouTubeVideo

In [ ]:
# Latest version of gensim
import gensim
gensim.__version__

'4.1.2'

In [ ]:
# Defining the function for preprocessing test dataset which will run after submitting the notebook

def tokenize_sentence(sentences, num_words=10000, maxlen=256, show=False): 

  # Creating the tokenizer, the num_words represents the vocabulary and assigning OOV token ( out of vocaculary ) for unknown tokenn
  # Which can arise if we input a sentence containing a words that tokenizer don't have in his vocabulary

  tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")


  tokenizer.fit_on_texts(sentences)
  
  # Getting the unique ID for each token
  word_index = tokenizer.word_index

  # Convert the senteces into vector
  sequences = tokenizer.texts_to_sequences(sentences)

  # Padding the vectors so that all vectors have the same length
  padded_sequences = pad_sequences(sequences, padding='post', truncating='pre', maxlen=maxlen)


  word_index = np.asarray(word_index)
  sequences = np.asarray(sequences)
  padded_sequences = np.asarray(padded_sequences)

  if show==True:
    console = Console()

    console.log("Word Index. A unique ID is assigned to each token.")
    console.log(word_index)
    console.log("---"*10)

    console.log("Sequences. senteces converted into vector.")
    console.log(np.array(sequences[0]))
    console.log("---"*10)

    console.log("Padded Sequences. Adding,( 0 in this case ) or removing elements to make all vectors in the samples same.")
    console.log(np.array(padded_sequences[0]))
    console.log("---"*10)



  return tokenizer, word_index, sequences, padded_sequences

# Training phase ⚙️

You can define your training code here. This sections will be skipped during evaluation.

## Downloading Dataset

Must be prety familar thing by now :) In case, here we are downloading the challange dataset using AIcrowd CLI 

In [ ]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/v8lvFNYrbzG2z76VDxvs58ogtqbzRNjoK7XEZp3tx54
API Key valid
Saved API Key successfully!


In [ ]:
# Downloading the Dataset
!mkdir data
!aicrowd dataset download --challenge nlp-feature-engineering-2 -j 3 -o data

# Donwloading programming language classification dataset for testing purposes
!mkdir programming-language-data
!aicrowd dataset download --challenge programming-language-classification -j 3 -o programming-language-data

data.csv: 100% 8.37k/8.37k [00:00<00:00, 260kB/s]
sample_submission.csv:   0% 0.00/121k [00:00<?, ?B/s]
test.csv:   0% 0.00/1.50M [00:00<?, ?B/s]

sample_submission.csv: 100% 121k/121k [00:00<00:00, 619kB/s]

test.csv: 100% 1.50M/1.50M [00:00<00:00, 2.89MB/s]


train.csv: 100% 7.71M/7.71M [00:00<00:00, 11.0MB/s]


### Reading Dataset

Reading the necessary files to train, validation & submit our results! 

We are also using [Programming Language Challange](https://www.aicrowd.com/challenges/ai-blitz-xii/problems/programming-language-classification) dataset for testing purposes.

In [ ]:
dataset = pd.read_csv("data/data.csv")
train_data = pd.read_csv("programming-language-data/train.csv")

dataset

,id,text,feature
0,0,Incels can't even get a pleb thing called sex....,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, ..."
1,1,Seriously? I couldn't even remember the dude's...,"[0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, ..."
2,2,Seeing [NAME] on Sunday and I'm so fucking stoked,"[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, ..."
3,3,"Your not winning their hearts, your just tortu...","[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, ..."
4,4,This sub needs more Poison memes.,"[1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, ..."
5,5,It is part of the political game unfortunately.,"[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, ..."
6,6,"That looks lovely, but what’s the mutant on th...","[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, ..."
7,7,Who will rid me of these meddlesome Golden Kni...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, ..."
8,8,We were all duped? I’ve seen through her from ...,"[0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, ..."
9,9,"""Lol, hold my craft beer.""","[0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, ..."


In [ ]:
introduction_file_doc = nlp(train_data['code'].values[0])
print ([token.text for token in introduction_file_doc])

['            ', 'var', 'result', '=', 'testObj1', '|', 'testObj2', ';', '\n\n             ', '//', 'Assert', '\n\n             ', 'Assert', '.', 'AreEqual(expected', ',', 'result', '.', 'ToString', '(', ')', ')', ';', '\n\n         ', '}', '\n\n         ', '[', 'TestCase(1', ',', '1', ',', '1', ',', '1', ',', '"', '1', '"', ')', ']', '\n\n         ', '[', 'TestCase(5', ',', '3', ',', '8', ',', '4', ',', '"', '0000', '"', ')', ']', '\n']


## Creating our Templete

So, with this `train_model` we are going to text the various differetn techniques and pare to see which works best!

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import xgboost
from sklearn.svm import LinearSVC
def train_model(X, y):

  # Splitting the dataset into training and testing,  also by using stratify, we are making sure to use the same class balance between training and testing. 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

  clf=xgboost.XGBClassifier(learning_rate=0.001,n_estimators=100,reg_alpha=0.03)
  #clf=LGBMClassifier(num_leaves=60,min_child_samples=5,max_depth=-1,learning_rate=0.1,reg_alpha=0.03)
  
  #clf=SGDClassifier(penalty='l2',learning_rate='adaptive',eta0=0.01)
  # Creating and training sklearn's Decision Tree Classifier Model 
  # clf = LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,  
  #                  intercept_scaling=1, l1_ratio=None, max_iter=200,  
  #                  multi_class='multinomial', n_jobs=-1, penalty='l2',  
  #                  random_state=0, solver='lbfgs', tol=0.0001, verbose=0,  
  #                  warm_start=False)
  clf.fit(X_train, y_train)

  # Getting the predictions form unseen (testing dataset)
  predictions = clf.predict(X_test)

  # Calcuating the metrics 
  f1 = f1_score(y_test, predictions, average='weighted')
  accuracy = accuracy_score(y_test, predictions)

  # Creating the table
  console = Console()
  result_table = Table(show_header=False, header_style="bold magenta")

  result_table.add_row("F1 Score", str(f1))
  result_table.add_row("Accuracy Score", str(accuracy))

  # Showing the table
  console.print(result_table)

  return f1, accuracy

## Simple Tokenization 🪙

Here, all what we are doing is splitting the senteces into tokens/words, and then assigning a unique id to each token, and here we go, we converted the text into a vector. We are also using padding to make sure all vectors are of `maxlen` which is 256. 

In [ ]:
def tokenize_sentence(sentences, num_words=10000, maxlen=256, show=False): 

  # Creating the tokenizer, the num_words represents the vocabulary and assigning OOV token ( out of vocaculary ) for unknown tokenn
  # Which can arise if we input a sentence containing a words that tokenizer don't have in his vocabulary

  tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")


  tokenizer.fit_on_texts(sentences)
  
  # Getting the unique ID for each token
  word_index = tokenizer.word_index

  # Convert the senteces into vector
  sequences = tokenizer.texts_to_sequences(sentences)

  # Padding the vectors so that all vectors have the same length
  padded_sequences = pad_sequences(sequences, padding='post', truncating='pre', maxlen=maxlen)


  word_index = np.asarray(word_index)
  sequences = np.asarray(sequences)
  padded_sequences = np.asarray(padded_sequences)

  if show==True:
    console = Console()

    console.log("Word Index. A unique ID is assigned to each token.")
    console.log(word_index)
    console.log("---"*10)

    console.log("Sequences. senteces converted into vector.")
    console.log(np.array(sequences[0]))
    console.log("---"*10)

    console.log("Padded Sequences. Adding,( 0 in this case ) or removing elements to make all vectors in the samples same.")
    console.log(np.array(padded_sequences[0]))
    console.log("---"*10)



  return tokenizer, word_index, sequences, padded_sequences

In [ ]:
# Sample Senteces
sample_sentences = dataset.iloc[0, 1].split(".")
sample_sentences

[
    "Incels can't even get a pleb thing called sex",
    ' Sex is for [NAME] apparently',
    ''
]

In [ ]:
_, _, _, _ = tokenize_sentence(sample_sentences, num_words=50, maxlen=16, show=True)

[10:20:41] Word Index. A unique ID is assigned to each     <ipython-input-13-034b9db2f918>:28
           token.

           {'<OOV>': 1, 'sex': 2, 'incels': 3, "can't": 4, <ipython-input-13-034b9db2f918>:29
           'even': 5, 'get': 6, 'a': 7, 'pleb': 8,                                           
           'thing': 9, 'called': 10, 'is': 11, 'for': 12,                                    
           'name': 13, 'apparently': 14}

           ------------------------------                  <ipython-input-13-034b9db2f918>:30

           Sequences. senteces converted into vector.      <ipython-input-13-034b9db2f918>:32

           [ 3  4  5  6  7  8  9 10  2]                    <ipython-input-13-034b9db2f918>:33

           ------------------------------                  <ipython-input-13-034b9db2f918>:34

           Padded Sequences. Adding,( 0 in this case ) or  <ipython-input-13-034b9db2f918>:36
           removing elements to make all vectors in the                                      
           samples same.

           [ 3  4  5  6  7  8  9 10  2  0  0  0  0  0  0   <ipython-input-13-034b9db2f918>:37
           0]

           ------------------------------                  <ipython-input-13-034b9db2f918>:38

In [ ]:
# Training the model using the vectors and the features

tokenizer, _, _, X = tokenize_sentence(train_data['code'].values)
y = train_data['language'].values

In [ ]:
print("Sentence : ", train_data['code'][2])
print("Simple Tokenizer : ", X[2])

Sentence :  /*

     Explanation :- a user gives a String (it can be incomplete uppercase or

         partial uppercase) and then the program would convert it into a

         complete(all characters in lower case) lower case string. The

Simple Tokenizer :  [ 862    8  377 1465    8   33   90  179   49    1 1531  109 3361 1531
   24  247    3  467  821  442   90  295    8 1489  123  450   15  372
  156  372  156   33    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0

In [ ]:
token_id_f1, token_id_accuracy = train_model(X, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.2680233576431608 ┃
│ Accuracy Score │ 0.3449484987946526 │
└────────────────┴────────────────────┘

In [ ]:
tokenizer, _, _, _ = tokenize_sentence(train_data['code'].values, num_words=256)
X = tokenizer.texts_to_matrix(train_data['code'].values,mode='freq')

In [ ]:
_f1,_accuracy = train_model(X, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.5278126775704207 ┃
│ Accuracy Score │ 0.544488275257506  │
└────────────────┴────────────────────┘

In [ ]:
tokenizer, _, _, _ = tokenize_sentence(train_data['code'].values, num_words=256)
X = tokenizer.texts_to_matrix(train_data['code'].values,mode='binary')

In [ ]:
_f1,_accuracy = train_model(X, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.5237139721267318 ┃
│ Accuracy Score │ 0.5408722331799255 │
└────────────────┴────────────────────┘

Now, the advantages of this method is that it is very simple, but one of the major disadvantages for this is it doesn't contain the "meaning" of the text, and mny next will also not be able to solve this issue, unzip word2vec  

## Bag of Words 🎒

In Bag of Words, instead of what we did in simple tokenization, just assiging a unique ID to each token, bag of words, does things a little different.

I find bag of words harder to understand with a text, so i find [this](https://youtu.be/UFtXy0KRxVI) video really helpful for understand bag of words in a more visual way. Be sure to watch it. 

In [ ]:
tokenizer, _, _, _ = tokenize_sentence(train_data['code'].values, num_words=256)
X = tokenizer.texts_to_matrix(train_data['code'].values)

In [ ]:
print("Sentence : ", train_data['code'][0])
print("BOW : ", X[0])

Sentence :              var result = testObj1 | testObj2;

             // Assert

             Assert.AreEqual(expected, result.ToString());

         }

         [TestCase(1, 1, 1, 1, "1")]

         [TestCase(5, 3, 8, 4, "0000")]

BOW :  [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [ ]:
bow_f1, bow_accuracy = train_model(X, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.5237139721267318 ┃
│ Accuracy Score │ 0.5408722331799255 │
└────────────────┴────────────────────┘

Yee! both of the metrics did increased! Advantages of Bag of words is that it's again, really simple, but it doesn't keep the same order of words in sentence and doesn't keep the meaning of the sentence.  

## Count Vectorization 🔢

Count Vectorization is also very similar to Bag of Wards but instead of one hot, it also include the count of each token in a sentece. 

In [ ]:
tokenizer, _, x, xx = tokenize_sentence(train_data['code'].values, num_words=256)
X = tokenizer.texts_to_matrix(train_data['code'].values, mode='count')

In [ ]:
print("Sentence : ", train_data['code'][2])
print("CV : ", X[2])

Sentence :  /*

     Explanation :- a user gives a String (it can be incomplete uppercase or

         partial uppercase) and then the program would convert it into a

         complete(all characters in lower case) lower case string. The

CV :  [ 0. 15.  0.  2.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.

In [ ]:
count_f1, count_accuracy = train_model(X, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.5273012040061048 ┃
│ Accuracy Score │ 0.5430637738330046 │
└────────────────┴────────────────────┘

In [ ]:
count_f1, count_accuracy = train_model(xx, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.3086611316667192  ┃
│ Accuracy Score │ 0.37409598948060485 │
└────────────────┴─────────────────────┘

## TF - IDF 📐

TF-IDF is **Term Frequency - Inverse Document Frequency**. So as we way in last section, about count frequency, that had a bit of flaw, for ex. tokens such a `is, are, the` are very common and will generally have bigger counts, but they don't ususally help for ex. classify whether the text is positive or negative. TF-IDF actually try to solve this issue. TF-IDF applies lower score to the common tokens and higher scores for more rarer tokens. 

In [ ]:
tokenizer, _, _, x = tokenize_sentence(train_data['code'].values, num_words=256)
X = tokenizer.texts_to_matrix(train_data['code'].values, mode='tfidf')

In [ ]:
print("Sentence : ", train_data['code'][0])
print("TF-IDF : ", X[0])

Sentence :              var result = testObj1 | testObj2;

             // Assert

             Assert.AreEqual(expected, result.ToString());

         }

         [TestCase(1, 1, 1, 1, "1")]

         [TestCase(5, 3, 8, 4, "0000")]

TF-IDF :  [ 0.         31.60460419  4.23447758  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          2.57234506  0.          0.          0.          0.
  0.          0.          0.          0.          2.88231493  2.92883682
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          5.82824057  0.          0.          0.
  0.          0.          0.     

In [ ]:
tfidf_f1, tfidf_accuracy = train_model(X, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.5273012040061048 ┃
│ Accuracy Score │ 0.5430637738330046 │
└────────────────┴────────────────────┘

In [ ]:
tfidf_f1, tfidf_accuracy = train_model(x, y)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ F1 Score       ┃ 0.3086611316667192  ┃
│ Accuracy Score │ 0.37409598948060485 │
└────────────────┴─────────────────────┘

# Prediction phase 🔎

Generating the features in test dataset. 

In [ ]:
test_dataset = pd.read_csv(AICROWD_DATASET_PATH)
test_dataset

,id,text,feature
0,0,Incels can't even get a pleb thing called sex....,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, ..."
1,1,Seriously? I couldn't even remember the dude's...,"[0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, ..."
2,2,Seeing [NAME] on Sunday and I'm so fucking stoked,"[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, ..."
3,3,"Your not winning their hearts, your just tortu...","[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, ..."
4,4,This sub needs more Poison memes.,"[1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, ..."
5,5,It is part of the political game unfortunately.,"[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, ..."
6,6,"That looks lovely, but what’s the mutant on th...","[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, ..."
7,7,Who will rid me of these meddlesome Golden Kni...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, ..."
8,8,We were all duped? I’ve seen through her from ...,"[0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, ..."
9,9,"""Lol, hold my craft beer.""","[0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, ..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',max_features=256)
X = vectorizer.fit_transform(train_data['code'].values)

In [ ]:
# from sklearn.cluster import KMeans
# true_k = 10
# model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
# model.fit(X)

KMeans(max_iter=100, n_clusters=10, n_init=1)

In [ ]:
# order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#X.shape

(10, 256)

In [ ]:
# for i in range(true_k):
#  print('Cluster %d:' % i),
#  for ind in order_centroids[i, :10]:
#    print(' %s' % terms[ind])

Cluster 0:
 node
 right
 left
 root
 null
 return
 parent
 data
 value
 key
Cluster 1:
 return
 number
 array
 true
 param
 list
 print
 value
 10
 false
Cluster 2:
 20
 10
 15
 30
 include
 17
 12
 test
 11
 50
Cluster 3:
 int
 return
 list
 def
 void
 array
 arr
 number
 new
 size
Cluster 4:
 public
 class
 void
 static
 test
 new
 int
 summary
 private
 namespace
Cluster 5:
 std
 cout
 endl
 vector
 int
 test
 double
 passed
 assert
 cin
Cluster 6:
 self
 def
 return
 node
 index
 data
 len
 str
 range
 list
Cluster 7:
 main
 printf
 int
 function
 return
 exit
 void
 test
 brief
 returns
Cluster 8:
 string
 return
 str
 text
 length
 int
 param
 new
 std
 input
Cluster 9:
 __main__
 __name__
 doctest
 testmod
 print
 import
 return
 solution
 main
 input


In [ ]:
# for i in test_dataset['text'].values:
#   print(i)
#   X = vectorizer.transform([i])
#   predicted = model.predict(X)
#   print(predicted)

Incels can't even get a pleb thing called sex. Sex is for [NAME] apparently.
[1]
Seriously? I couldn't even remember the dude's name until about halfway through the season honestly.
[1]
Seeing [NAME] on Sunday and I'm so fucking stoked
[1]
Your not winning their hearts, your just torturing them and they end up hating you.
[1]
This sub needs more Poison memes.
[1]
It is part of the political game unfortunately.
[1]
That looks lovely, but what’s the mutant on the top?
[1]
Who will rid me of these meddlesome Golden Knights?
[1]
We were all duped? I’ve seen through her from day 1, even posted about how fake she is
[1]
"Lol, hold my craft beer."
[1]


In [ ]:
# len(vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


256

In [ ]:
# So, let's do a simple tokenization and generate the features!
# tokenizer, _, _, _ = tokenize_sentence(test_dataset['text'].values, num_words=256)
# #tokenizer.texts_to_matrix()
# X = tokenizer.texts_to_matrix(test_dataset['text'].values, mode='tfidf')

X=vectorizer.transform(test_dataset['text'].values)
X=X.toarray()

#_, _, _, X = tokenize_sentence(test_dataset['text'].values)

for index, row in tqdm(test_dataset.iterrows()):
  test_dataset.iloc[index, 2] = str(X[index])

# for index, row in tqdm(test_dataset.iterrows()):
#   test_dataset.iloc[index, 2] = str(X[index].tolist())

test_dataset

0it [00:00, ?it/s]

,id,text,feature
0,0,Incels can't even get a pleb thing called sex....,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1,1,Seriously? I couldn't even remember the dude's...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2,2,Seeing [NAME] on Sunday and I'm so fucking stoked,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3,3,"Your not winning their hearts, your just tortu...",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4,4,This sub needs more Poison memes.,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
5,5,It is part of the political game unfortunately.,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
6,6,"That looks lovely, but what’s the mutant on th...",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7,7,Who will rid me of these meddlesome Golden Kni...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
8,8,We were all duped? I’ve seen through her from ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
9,9,"""Lol, hold my craft beer.""",[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [ ]:
# Saving the sample submission
test_dataset.to_csv(os.path.join(AICROWD_OUTPUTS_PATH,'submission.csv'), index=False)

# Submit to AIcrowd 🚀

**Note : Please save the notebook before submitting it (Ctrl + S)**

In [ ]:
%aicrowd notebook submit --assets-dir $AICROWD_ASSETS_DIR --challenge nlp-feature-engineering-2 --no-verify

Using notebook: NLP Feature Engineering #2 Baseline for submission...
Scrubbing API keys from the notebook...


Output()

╭─────────────────────────╮                                                       
                                                       │ Successfully submitted! │                                                       
                                                       ╰─────────────────────────╯

                                                             Important links                                                             
┌──────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/nlp-feature-engineering-2/submissions/170715              │
│                  │                                                                                                                    │
│  All submissions │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/nlp-feature-engineering-2/submissions?my_submissions=true │
│                  │                                                                                                                    │
│      Leaderboard │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/nlp-feature-engineering-2/leaderboards                    │
│                  │                                                                                                                    │
│ Discussion forum │ https://discourse.aicrowd.com/c/ai-blitz-xii                                                                       │
│                  │                                                                                                                    │
│   Challenge page │ https://www.aicrowd.com/challenges/ai-blitz-xii/problems/nlp-feature-engineering-2                                 │
└──────────────────┴────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘